### 3. A grading assistant
Teachers in general have a lot of administrations to do and one of those things is grading. Can we create a simple grade assistant to assist a Swedish teacher in grading? This exercise focuses a lot in prompt engineering and afterwards to postprocess the output using Pydantic.

a) Go into this page with examples of students answers to a particular question. Copy some example texts and paste it into files with names like student_text_1.txt, student_text_2.txt.

b) Read these data into python and tell your LLM to grade them.

c) Prompt to get an output of fields proposed_grade, motivation and improvements.

d) Now validate this with pydantic model

e) Output a folder with the following txt files: proposed_grade.txt, motivation.txt and improvements.txt

f) Go into skolverket for Svenska 1 and copy "Betygskriterier" for "Svenska 1". These are the criterias for the different grades. Paste this into a file called criterias.txt.

g) Now repeat b)-e) but with the criterias in your prompt as well. Can you see any differences in the outputs?

h) Can you improve the output quality by providing few shot examples?

In [ ]:
from pathlib import Path

#Arbetskatalogen där skriptet skript körs
current_dir = Path.cwd()

#Pekar på mappen där texterna ligger
DATA_PATH = current_dir / "exercise_data"

In [ ]:
#Debug-utskrift pga problem att hitta filen...

from pathlib import Path

print("CWD:", Path.cwd())
DATA_PATH = (Path.cwd() / "exercise_data").resolve()
print("DATA_PATH:", DATA_PATH, "exists:", DATA_PATH.exists())
print("Filer i DATA_PATH:", [p.name for p in DATA_PATH.glob("*.txt")])


CWD: c:\Users\susan\KodPython\ai_engineering_susanne_wenblad\exercises\Exercise3_pydantic
DATA_PATH: C:\Users\susan\KodPython\ai_engineering_susanne_wenblad\exercises\Exercise3_pydantic\exercise_data exists: True
Filer i DATA_PATH: ['student1.txt', 'student2.txt']


In [ ]:
#Funktion för att läsa in filerna 

def read_txt(filename: str) -> str:
    filepath = DATA_PATH / filename
    with filepath.open("r", encoding="utf-8") as f:
        return f.read()


In [ ]:
#Variabler för att lagra de två studenttexterna

text1 = read_txt("student1.txt")
text2 = read_txt("student2.txt")

In [ ]:
# Slår ihop hämtning av filer och lägger texterna i en lista

DATA_PATH = (Path.cwd() / "exercise_data").resolve()

def read_all_txts(path: Path):
    texts = []
    for file in sorted(path.glob("*.txt")): 
        with file.open("r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts

all_texts = read_all_txts(DATA_PATH)

print(f"Hittade {len(all_texts)} texter.")
print(all_texts[0][:200])  


Hittade 2 texter.
 Inför allas blickar
 En av de större andledningarna att man är tal rädd är förmodligen att man är 
rädd för att folket som lyssnar på kommer göra narr av en. När man är ensam 
på en scen eller i klas


In [ ]:
#Funktion för att läsa in texterna med namn och spara i en dict
def read_all_texts_with_names(path: Path):
    texts = {}
    for file in sorted(path.glob("*.txt")):
        with file.open("r", encoding="utf-8") as f:
            texts[file.stem] = f.read()
    return texts


texts_dict = read_all_texts_with_names(DATA_PATH)

print(texts_dict.keys())  
print(texts_dict["student1"][:200])

dict_keys(['student1', 'student2'])
 Inför allas blickar
 En av de större andledningarna att man är tal rädd är förmodligen att man är 
rädd för att folket som lyssnar på kommer göra narr av en. När man är ensam 
på en scen eller i klas


b) Read these data into python and tell your LLM to grade them.

In [ ]:
#Koppla Gemini till python

from google import genai

client = genai.Client()

In [ ]:
#Funktion för promptning samt test för att säkerställa att funktionen fungerar som den ska

def ask_LLM(prompt): 
    response = client.models.generate_content(
        model = "gemini-2.5-flash", 
        contents = prompt
    )
    return response.text

ask_LLM("Skriv tre svenska flicknamn")

'Här är tre svenska flicknamn:\n\n1.  **Linnea**\n2.  **Maja**\n3.  **Astrid**'

c) Prompt to get an output of fields proposed_grade, motivation and improvements.



In [ ]:
#Läs in studenttexterna med dict-funktionen 

texts_dict = read_all_texts_with_names(DATA_PATH)
student1_text = texts_dict["student1"]


#Skapa prompten
prompt_for_grading = f"""
Du är en svensk lärare och ska betygsätta en uppgift.
Här är studentens svar:
---
{student1_text}
---

Bedöm studentens svar och ge en sammanfattning med följande fält:
proposed_grade: [Ditt föreslagna betyg, t.ex. A, B, C, D, E, F]
motivation: [En kort motivering för betyget]
improvements: [Förslag på hur studenten kan förbättra sitt svar]

Svaret ska vara kortfattat. Max 10 rader
"""

#Anropar funktionen och sparar resultatet
llm_response = ask_LLM(prompt_for_grading)

#Skriver ut resultatet från LLM:en
print(llm_response)


proposed_grade: D

motivation: Studenten visar en grundläggande förståelse för ämnet och gör ett försök att integrera en extern källa med egna tankar. En viss struktur finns i texten. Däremot är språket tyvärr bristfälligt med många stavfel, grammatiska fel samt inkonsekvenser i meningsbyggnad och interpunktion. Dessa brister försvårar läsningen och drar ner textens akademiska kvalitet betydligt.

improvements: Fokusera på att förbättra grundläggande språkkunskaper. Arbeta med stavning (särskilt sammansatta ord som "klassrummet", "jobbskäl", "självförtroende"), grammatik (t.ex. formlära som "van" istället för "vann", "utstött" istället för "utstöt", och konsekvens i personval). Öva på interpunktion och läs noga korrektur för att upptäcka och rätta fel innan inlämning.


d) Now validate this with pydantic model



In [17]:
from pydantic import BaseModel

#Modell för att validera utdata från LLM

class GradeOutput(BaseModel):
    proposed_grade: str
    motivation: str
    improvements: str




In [ ]:
#Funktion för att parsa datan till en dict
def parse_llm_response(text: str) -> dict:
    """Parsar en textsträng från LLM till en dictionary."""
    parsed_data = {}
    lines = text.strip().split('\n')
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            parsed_data[key.strip()] = value.strip()
    return parsed_data

In [21]:

# 1. Parsa strängen  till en dictionary
parsed_data = parse_llm_response(llm_response)

# 2. Validera med Pydantic-modellen
try:
    validated_output = GradeOutput(**parsed_data)
    print("Utdata validerad med Pydantic!")
    print(validated_output.model_dump_json(indent=2))

except Exception as e:
    print(f"Validering misslyckades: {e}")

Utdata validerad med Pydantic!
{
  "proposed_grade": "D",
  "motivation": "Studenten visar en grundläggande förståelse för ämnet och gör ett försök att integrera en extern källa med egna tankar. En viss struktur finns i texten. Däremot är språket tyvärr bristfälligt med många stavfel, grammatiska fel samt inkonsekvenser i meningsbyggnad och interpunktion. Dessa brister försvårar läsningen och drar ner textens akademiska kvalitet betydligt.",
  "improvements": "Fokusera på att förbättra grundläggande språkkunskaper. Arbeta med stavning (särskilt sammansatta ord som \"klassrummet\", \"jobbskäl\", \"självförtroende\"), grammatik (t.ex. formlära som \"van\" istället för \"vann\", \"utstött\" istället för \"utstöt\", och konsekvens i personval). Öva på interpunktion och läs noga korrektur för att upptäcka och rätta fel innan inlämning."
}


e) Output a folder with the following txt files: proposed_grade.txt, motivation.txt and improvements.txt



In [ ]:
from pathlib import Path

output_folder = DATA_PATH.parent / "output_data"
output_folder.mkdir(exist_ok=True)

In [25]:
# Sparar betyget
with open(output_folder / 'proposed_grade.txt', 'w', encoding='utf-8') as f:
    f.write(validated_output.proposed_grade)

# Sparar motiveringen
with open(output_folder / 'motivation.txt', 'w', encoding='utf-8') as f:
    f.write(validated_output.motivation)

# Sparar förbättringsförslagen
with open(output_folder / 'improvements.txt', 'w', encoding='utf-8') as f:
    f.write(validated_output.improvements)

print("Bedömningen har sparats i mappen 'output_data'.")

Bedömningen har sparats i mappen 'output_data'.


f) Go into skolverket for Svenska 1 and copy "Betygskriterier" for "Svenska 1". These are the criterias for the different grades. Paste this into a file called criterias.txt.



g) Now repeat b)-e) but with the criterias in your prompt as well. Can you see any differences in the outputs?



h) Can you improve the output quality by providing few shot examples?